## DS 2002: Midterm Project
#### Eva Terry, bzt4em
Before getting to this notebook, I first ran the mysqlsampledatabase.sql script in MySQL which creates and populates the classicmodels schema. This script is included in my repository. From there, I exported a the orders and order details dimensions from classicmodels as json files and exported the customers dimension as a csv on the file system (these three files are in the repository). After the classicmodels schema is set up along with a blank schema for classic_dw, I executed the steps below. This included running SQL scripts that create dim_date and dim_products from a relational database. 

In addition to the date dimension, I have a dimension for customers, products, order details, and orders. The orders and order details dimensions were from a NoSQL database (MongoDB), the customers dimension came from the file system (read in as a csv), and the products dimension was created in a relational database (MySQL). I created a fact table with information from all five dimensions called df_fact_orders, which is queried at the bottom of this notebook. 

#### Import the Necessary Libraries

In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.2


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL & Databases with which You'll be Working 

In [3]:
mysql_uid = "root"
mysql_pwd = "Passw0rd123"
mysql_hostname = "localhost"

conn_str = {"local" : f"mongodb://localhost:27017/"
}

src_dbname = "classic_purchasing"
dst_dbname = "classic_dw"

print(f"Local Connection String: {conn_str['local']}")

Local Connection String: mongodb://localhost:27017/


#### Define Functions for Getting Data From and Setting Data Into Databases

In [4]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create a Destination Database for the New Dataware House

In [5]:
sql_conn_str = f"mysql+pymysql://{mysql_uid}:{mysql_pwd}@{mysql_hostname}"
sqlEngine = create_engine(sql_conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#### Create the Products Dimension Using a Relational Database (MySQL)
Note that this script is also given by dim_products_query.sql which is included in my repository.

In [6]:
create_dim_products_sql_queries = [
    "USE classic_dw;",
    "DROP TABLE IF EXISTS dim_products;",
    """
    CREATE TABLE `dim_products` (
      `productCode` varchar(15) NOT NULL,
      `productName` varchar(70) NOT NULL,
      `productLine` varchar(50) NOT NULL,
      `productScale` varchar(10) NOT NULL,
      `productVendor` varchar(50) NOT NULL,
      `productDescription` text NOT NULL,
      `quantityInStock` smallint NOT NULL,
      `buyPrice` decimal(10,2) NOT NULL,
      `MSRP` decimal(10,2) NOT NULL,
      PRIMARY KEY (`productCode`),
      KEY `productLine` (`productLine`)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;
    """,
    """
    INSERT INTO `classic_dw`.`dim_products`
    (`productCode`,
    `productName`,
    `productLine`,
    `productScale`,
    `productVendor`,
    `productDescription`,
    `quantityInStock`,
    `buyPrice`,
    `MSRP`)
    SELECT `products`.`productCode`,
        `products`.`productName`,
        `products`.`productLine`,
        `products`.`productScale`,
        `products`.`productVendor`,
        `products`.`productDescription`,
        `products`.`quantityInStock`,
        `products`.`buyPrice`,
        `products`.`MSRP`
    FROM `classicmodels`.`products`;
    """
]

for query in create_dim_products_sql_queries:
    sqlEngine.execute(query)

#### Create and Populate the Date Dimension
Note that this script is also given by Create_Populate_Dim_Date.sql which is included in my repository.

In [7]:
create_dim_date_sql_queries = [
    "USE classic_dw;",
    "DROP TABLE IF EXISTS dim_date;",
    """CREATE TABLE dim_date (
         date_key INT NOT NULL,
         full_date DATE NULL,
         date_name CHAR(11) NOT NULL,
         date_name_us CHAR(11) NOT NULL,
         date_name_eu CHAR(11) NOT NULL,
         day_of_week TINYINT NOT NULL,
         day_name_of_week CHAR(10) NOT NULL,
         day_of_month TINYINT NOT NULL,
         day_of_year SMALLINT NOT NULL,
         weekday_weekend CHAR(10) NOT NULL,
         week_of_year TINYINT NOT NULL,
         month_name CHAR(10) NOT NULL,
         month_of_year TINYINT NOT NULL,
         is_last_day_of_month CHAR(1) NOT NULL,
         calendar_quarter TINYINT NOT NULL,
         calendar_year SMALLINT NOT NULL,
         calendar_year_month CHAR(10) NOT NULL,
         calendar_year_qtr CHAR(10) NOT NULL,
         fiscal_month_of_year TINYINT NOT NULL,
         fiscal_quarter TINYINT NOT NULL,
         fiscal_year INT NOT NULL,
         fiscal_year_month CHAR(10) NOT NULL,
         fiscal_year_qtr CHAR(10) NOT NULL,
         PRIMARY KEY (date_key)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8;""",
    """CREATE PROCEDURE PopulateDateDimension(BeginDate DATETIME, EndDate DATETIME)
       BEGIN
           DECLARE LastDayOfMon CHAR(1);
           DECLARE FiscalYearMonthsOffset INT;
           DECLARE DateCounter DATETIME;
           DECLARE FiscalCounter DATETIME;
           SET FiscalYearMonthsOffset = 6;
           SET DateCounter = BeginDate;
           WHILE DateCounter <= EndDate DO
               SET FiscalCounter = DATE_ADD(DateCounter, INTERVAL FiscalYearMonthsOffset MONTH);
               IF MONTH(DateCounter) = MONTH(DATE_ADD(DateCounter, INTERVAL 1 DAY)) THEN
                   SET LastDayOfMon = 'N';
               ELSE
                   SET LastDayOfMon = 'Y';
               END IF;
               INSERT INTO dim_date
                   (date_key, full_date, date_name, date_name_us, date_name_eu,
                    day_of_week, day_name_of_week, day_of_month, day_of_year,
                    weekday_weekend, week_of_year, month_name, month_of_year,
                    is_last_day_of_month, calendar_quarter, calendar_year,
                    calendar_year_month, calendar_year_qtr, fiscal_month_of_year,
                    fiscal_quarter, fiscal_year, fiscal_year_month, fiscal_year_qtr)
               VALUES
                   ((YEAR(DateCounter) * 10000) + (MONTH(DateCounter) * 100) + DAY(DateCounter),
                   DateCounter,
                   CONCAT(CAST(YEAR(DateCounter) AS CHAR(4)),'/', DATE_FORMAT(DateCounter,'%%%%m'),'/', DATE_FORMAT(DateCounter,'%%%%d')),
                   CONCAT(DATE_FORMAT(DateCounter,'%%%%m'),'/', DATE_FORMAT(DateCounter,'%%%%d'),'/', CAST(YEAR(DateCounter) AS CHAR(4))),
                   CONCAT(DATE_FORMAT(DateCounter,'%%%%d'),'/', DATE_FORMAT(DateCounter,'%%%%m'),'/', CAST(YEAR(DateCounter) AS CHAR(4))),
                   DAYOFWEEK(DateCounter),
                   DAYNAME(DateCounter),
                   DAYOFMONTH(DateCounter),
                   DAYOFYEAR(DateCounter),
                   CASE DAYNAME(DateCounter)
                       WHEN 'Saturday' THEN 'Weekend'
                       WHEN 'Sunday' THEN 'Weekend'
                       ELSE 'Weekday'
                   END,
                   WEEKOFYEAR(DateCounter),
                   MONTHNAME(DateCounter),
                   MONTH(DateCounter),
                   LastDayOfMon,
                   QUARTER(DateCounter),
                   YEAR(DateCounter),
                   CONCAT(CAST(YEAR(DateCounter) AS CHAR(4)),'-',DATE_FORMAT(DateCounter,'%%%%m')),
                   CONCAT(CAST(YEAR(DateCounter) AS CHAR(4)),'Q',QUARTER(DateCounter)),
                   MONTH(FiscalCounter),
                   QUARTER(FiscalCounter),
                   YEAR(FiscalCounter),
                   CONCAT(CAST(YEAR(FiscalCounter) AS CHAR(4)),'-',DATE_FORMAT(FiscalCounter,'%%%%m')),
                   CONCAT(CAST(YEAR(FiscalCounter) AS CHAR(4)),'Q',QUARTER(FiscalCounter))
               );
               SET DateCounter = DATE_ADD(DateCounter, INTERVAL 1 DAY);
           END WHILE;
       END;""",
    "CALL PopulateDateDimension('2000/01/01', '2010/12/31');",
    "SELECT MIN(full_date) as BeginDate, MAX(full_date) as EndDate FROM dim_date;"
]



for query in create_dim_date_sql_queries:
    sqlEngine.execute(query)

#### Populating MongoDB with Order and Order Details Data 

In [8]:
client = pymongo.MongoClient(conn_str["local"])
db = client[src_dbname]

# Gets the path of the Current Working Directory for this Notebook, and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'project_data')

json_files = {"order_details" : 'classic_order_details.json', 
              "orders": 'classic_orders.json'
             }

for file in json_files:
    db.drop_collection(file)
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        #print(f"{file} was successfully loaded.")

        
client.close()      

### 1: Creating the Customers, Orders, and Order Details Dimensions
#### Reading in From the File System to Create df_customers

In [9]:
df_customers = pd.read_csv(os.path.join(data_dir, 'customers.csv'))
print(df_customers.head(2))
# read_csv exports a dataframe, so no need to convert into dataframe below

   customerNumber        customerName contactLastName contactFirstName  \
0             103   Atelier graphique         Schmitt          Carine    
1             112  Signal Gift Stores            King             Jean   

        phone     addressLine1 addressLine2       city state postalCode  \
0  40.32.2555   54, rue Royale          NaN     Nantes   NaN      44000   
1  7025551838  8489 Strong St.          NaN  Las Vegas    NV      83030   

  country  salesRepEmployeeNumber  creditLimit  
0  France                  1370.0      21000.0  
1     USA                  1166.0      71800.0  


#### Extracting Data from the Source MongoDB Collections (NoSQL database) Into the Orders and Order DDataFrames

In [10]:
query = {} # Select all elements (columns), and all documents (rows).
collection = "order_details"

df_order_details = get_mongo_dataframe(conn_str['local'], src_dbname, collection, query)
df_order_details.head(2)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2


In [11]:
query = {} # Select all elements (columns), and all documents (rows).
collection = "orders"

df_orders = get_mongo_dataframe(conn_str['local'], src_dbname, collection, query)  # Specify 'atlas', or 'local'
df_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128


#### Getting the Data from the Date Dimension

In [12]:
sql_dim_date = "SELECT date_key, full_date FROM classic_dw.dim_date;"
df_dim_date = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date


#### Getting the Datafor the Product Dimension
Before running this section, I ran the dim_products_query.sql to create the dimension in a MySQL Relational Database. Note that in order for this script to be successful, the classicmodels schema must have already been created using mysqlsampledatabase.sql. It is also necessary to have a schema for classic_dw, which is outlined above. 

In [13]:
sql_dim_products = "SELECT * FROM classic_dw.dim_products;"
df_dim_products = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_products)
df_dim_products.head(2)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


##### Looking up the Surrogate Primary Key (date_key) that Corresponds to the orderDate Column

In [14]:
df_dim_orderDate = df_dim_date.rename(columns={"date_key" : "orderDateKey", "full_date" : "orderDate"})
df_orders.orderDate = df_orders.orderDate.astype('datetime64[ns]').dt.date
df_orders = pd.merge(df_orders, df_dim_orderDate, on='orderDate', how='left')
df_orders.drop(['orderDate'], axis=1, inplace=True)
df_orders.head(2)

,orderNumber,requiredDate,shippedDate,status,comments,customerNumber,orderDateKey
0,10100,2003-01-13,2003-01-10,Shipped,None,363,NaN
1,10101,2003-01-18,2003-01-11,Shipped,Check on availability.,128,NaN


#### Inserting Primary Keys for the Order, Order Details, and Customers Dataframes

In [15]:
# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_orders.insert(0, "orderKey", range(1, df_orders.shape[0]+1))
df_orders.head(2)

,orderKey,orderNumber,requiredDate,shippedDate,status,comments,customerNumber,orderDateKey
0,1,10100,2003-01-13,2003-01-10,Shipped,None,363,NaN
1,2,10101,2003-01-18,2003-01-11,Shipped,Check on availability.,128,NaN


In [16]:
df_order_details.insert(0, "orderDetailsKey", range(1, df_order_details.shape[0]+1))
df_order_details.head(2)

,orderDetailsKey,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,1,10100,S18_1749,30,136.00,3
1,2,10100,S18_2248,50,55.09,2


In [17]:
df_customers.insert(0, "customerKey", range(1, df_customers.shape[0] + 1))
df_customers.head(2)

,customerKey,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800.0


#### Creating New Tables for the Orders, Order Details, and Customers Transformed Dataframe and Loading them into the Classic Data Warehouse

In [18]:
dataframe = df_orders
table_name = 'dim_orders'
primary_key = 'orderKey'
db_operation = "insert"

set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [19]:
dataframe = df_order_details
table_name = 'dim_order_details'
primary_key = 'orderDetailsKey'
db_operation = "insert"

set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [20]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customerKey'
db_operation = "insert"

set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### Validate that the Orders, Order Details and Customers Dimension Tables were Created.

In [21]:
sql_orders = "SELECT * FROM classic_dw.dim_orders;"
df_dim_orders = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_orders)
df_dim_orders.head(2)

,orderKey,orderNumber,requiredDate,shippedDate,status,comments,customerNumber,orderDateKey
0,1,10100,2003-01-13,2003-01-10,Shipped,None,363,None
1,2,10101,2003-01-18,2003-01-11,Shipped,Check on availability.,128,None


In [22]:
sql_order_details = "SELECT * FROM classic_dw.dim_order_details;"
df_dim_order_details = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_order_details)
df_dim_order_details.head(2)

,orderDetailsKey,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,1,10100,S18_1749,30,136.00,3
1,2,10100,S18_2248,50,55.09,2


In [23]:
sql_customers = "SELECT * FROM classic_dw.dim_customers;"
df_dim_customers = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_customers)
df_dim_customers.head(2)

,customerKey,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


### 2: Creating and Populating the Fact Table (df_fact_orders)

#### Starting to Create the Fact Orders Table by Merging df_orders with df_order_details

In [24]:
df_fact_orders = pd.merge(df_orders, df_order_details, on='orderNumber', how='right')
df_fact_orders.head(2)

,orderKey,orderNumber,requiredDate,shippedDate,status,comments,customerNumber,orderDateKey,orderDetailsKey,productCode,quantityOrdered,priceEach,orderLineNumber
0,1,10100,2003-01-13,2003-01-10,Shipped,None,363,NaN,1,S18_1749,30,136.00,3
1,1,10100,2003-01-13,2003-01-10,Shipped,None,363,NaN,2,S18_2248,50,55.09,2


#### Looking Up the DateKeys from the Date Dimension Table.

In [25]:
# Looking up the Surrogate Primary Key (date_key) that Corresponds to the "requiredDate" Column.
df_dim_required_date = df_dim_date.rename(columns={"date_key" : "requiredDateKey", "full_date" : "requiredDate"})
df_fact_orders.requiredDate = df_fact_orders.requiredDate.astype('datetime64[ns]').dt.date
df_fact_orders = pd.merge(df_fact_orders, df_dim_required_date, on='requiredDate', how='left')
df_fact_orders.drop(['requiredDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderKey,orderNumber,shippedDate,status,comments,customerNumber,orderDateKey,orderDetailsKey,productCode,quantityOrdered,priceEach,orderLineNumber,requiredDateKey
0,1,10100,2003-01-10,Shipped,None,363,NaN,1,S18_1749,30,136.00,3,NaN
1,1,10100,2003-01-10,Shipped,None,363,NaN,2,S18_2248,50,55.09,2,NaN


In [26]:
# Looking up the Surrogate Primary Key (date_key) that Corresponds to the "shippedDate" Column.
df_dim_shippedDate = df_dim_date.rename(columns={"date_key" : "shippedDateKey", "full_date" : "shippedDate"})
df_fact_orders.shippedDate = df_fact_orders.shippedDate.astype('datetime64[ns]').dt.date
df_fact_orders = pd.merge(df_fact_orders, df_dim_shippedDate, on='shippedDate', how='left')
df_fact_orders.drop(['shippedDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderKey,orderNumber,status,comments,customerNumber,orderDateKey,orderDetailsKey,productCode,quantityOrdered,priceEach,orderLineNumber,requiredDateKey,shippedDateKey
0,1,10100,Shipped,None,363,NaN,1,S18_1749,30,136.00,3,NaN,NaN
1,1,10100,Shipped,None,363,NaN,2,S18_2248,50,55.09,2,NaN,NaN


#### Adding a New Column for the Total Price of the Order

In [27]:
df_fact_orders['orderTotalPrice'] = df_fact_orders['quantityOrdered'] * df_fact_orders['priceEach']
df_fact_orders.head(2)

,orderKey,orderNumber,status,comments,customerNumber,orderDateKey,orderDetailsKey,productCode,quantityOrdered,priceEach,orderLineNumber,requiredDateKey,shippedDateKey,orderTotalPrice
0,1,10100,Shipped,None,363,NaN,1,S18_1749,30,136.00,3,NaN,NaN,4080.0
1,1,10100,Shipped,None,363,NaN,2,S18_2248,50,55.09,2,NaN,NaN,2754.5


#### Establishing Foreign Key Relationships between the Fact Table and the Customer and Product Dimensions

In [28]:
# getting the keys to link the customer dimension
sql_dim_customers = "SELECT customerKey, customerName, customerNumber FROM classic_dw.dim_customers;"
df_dim_customers_selected_columns = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_customers)
df_dim_customers_selected_columns.head(2)

,customerKey,customerName,customerNumber
0,1,Atelier graphique,103
1,2,Signal Gift Stores,112


In [29]:
# Modify 'df_fact_orders' by merging it with the customers dataframe on the 'customerNumber' column
df_fact_orders = pd.merge(df_fact_orders, df_dim_customers_selected_columns, on='customerNumber', how='inner')
df_fact_orders.drop(['customerNumber'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderKey,orderNumber,status,comments,orderDateKey,orderDetailsKey,productCode,quantityOrdered,priceEach,orderLineNumber,requiredDateKey,shippedDateKey,orderTotalPrice,customerKey,customerName
0,1,10100,Shipped,None,NaN,1,S18_1749,30,136.00,3,NaN,NaN,4080.0,86,Online Diecast Creations Co.
1,1,10100,Shipped,None,NaN,2,S18_2248,50,55.09,2,NaN,NaN,2754.5,86,Online Diecast Creations Co.


In [30]:
# getting the keys to link the product dimension
sql_dim_products = "SELECT productCode, productName FROM classic_dw.dim_products;"
df_dim_products_selected_columns = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_products)
df_dim_products_selected_columns.head(2)

,productCode,productName
0,S10_1678,1969 Harley Davidson Ultimate Chopper
1,S10_1949,1952 Alpine Renault 1300


In [31]:
# Modify 'df_fact_orders' by merging it with the product dataframe on the 'productCode' column
df_fact_orders = pd.merge(df_fact_orders, df_dim_products_selected_columns, on='productCode')
df_fact_orders.drop('productCode', axis=1, inplace=True)
df_fact_orders.head(2)

,orderKey,orderNumber,status,comments,orderDateKey,orderDetailsKey,quantityOrdered,priceEach,orderLineNumber,requiredDateKey,shippedDateKey,orderTotalPrice,customerKey,customerName,productName
0,1,10100,Shipped,None,NaN,1,30,136.0,3,NaN,NaN,4080.0,86,Online Diecast Creations Co.,1917 Grand Touring Sedan
1,74,10173,Shipped,Cautious optimism. We have happy customers her...,NaN,661,24,168.3,13,NaN,NaN,4039.2,57,Rovelli Gifts,1917 Grand Touring Sedan


#### Adding a Primary Key to the Order Fact Table

In [32]:
df_fact_orders.insert(0, "factOrdersKey", range(1, df_fact_orders.shape[0]+1))
df_fact_orders.head(2)

,factOrdersKey,orderKey,orderNumber,status,comments,orderDateKey,orderDetailsKey,quantityOrdered,priceEach,orderLineNumber,requiredDateKey,shippedDateKey,orderTotalPrice,customerKey,customerName,productName
0,1,1,10100,Shipped,None,NaN,1,30,136.0,3,NaN,NaN,4080.0,86,Online Diecast Creations Co.,1917 Grand Touring Sedan
1,2,74,10173,Shipped,Cautious optimism. We have happy customers her...,NaN,661,24,168.3,13,NaN,NaN,4039.2,57,Rovelli Gifts,1917 Grand Touring Sedan


#### Transforming the Fact Orders Table by Getting Rid of Unnecessary Columns

In [33]:
df_fact_orders.drop(['comments', 'orderLineNumber'], axis=1, inplace=True)
df_fact_orders.head(2)

,factOrdersKey,orderKey,orderNumber,status,orderDateKey,orderDetailsKey,quantityOrdered,priceEach,requiredDateKey,shippedDateKey,orderTotalPrice,customerKey,customerName,productName
0,1,1,10100,Shipped,NaN,1,30,136.0,NaN,NaN,4080.0,86,Online Diecast Creations Co.,1917 Grand Touring Sedan
1,2,74,10173,Shipped,NaN,661,24,168.3,NaN,NaN,4039.2,57,Rovelli Gifts,1917 Grand Touring Sedan


#### Transforming the Fact Orders Table by Renaming and Reordering Columns

In [34]:
# Rename columns
column_name_map = {"status" : "orderStatus",
                   "priceEach" : "unitPrice",
                  }

df_fact_orders.rename(columns=column_name_map, inplace=True)

# Reorder the Columns
ordered_columns = ['factOrdersKey', 'orderNumber','orderStatus'
                   ,'customerName','productName'
                   ,'unitPrice','quantityOrdered','orderTotalPrice', 
                  'orderKey', 'orderDateKey', 'orderDetailsKey', 'requiredDateKey', 'shippedDateKey', 
                  'customerKey']

df_fact_orders = df_fact_orders[ordered_columns]
df_fact_orders.head(2)

,factOrdersKey,orderNumber,orderStatus,customerName,productName,unitPrice,quantityOrdered,orderTotalPrice,orderKey,orderDateKey,orderDetailsKey,requiredDateKey,shippedDateKey,customerKey
0,1,10100,Shipped,Online Diecast Creations Co.,1917 Grand Touring Sedan,136.0,30,4080.0,1,NaN,1,NaN,NaN,86
1,2,10173,Shipped,Rovelli Gifts,1917 Grand Touring Sedan,168.3,24,4039.2,74,NaN,661,NaN,NaN,57


####  Loading the Fact Table / Transformed Data into classic_dw in MySQL

In [35]:
dataframe = df_fact_orders
table_name = 'fact_orders'
primary_key = 'factOrdersKey'
db_operation = "insert"

set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, dataframe, table_name, primary_key, db_operation)

####  Validating that the New Fact Table was Created and is in MySQL

In [36]:
sql_fact_orders = "SELECT * FROM classic_dw.fact_orders;"
df_fact_orders_validate = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_fact_orders)
df_fact_orders_validate.head(2)

,factOrdersKey,orderNumber,orderStatus,customerName,productName,unitPrice,quantityOrdered,orderTotalPrice,orderKey,orderDateKey,orderDetailsKey,requiredDateKey,shippedDateKey,customerKey
0,1,10100,Shipped,Online Diecast Creations Co.,1917 Grand Touring Sedan,136.0,30,4080.0,1,None,1,None,None,86
1,2,10173,Shipped,Rovelli Gifts,1917 Grand Touring Sedan,168.3,24,4039.2,74,None,661,None,None,57


### 3: SQL Queries

- Selecting data from the products dimension, customer dimension, and fact table
- Using SUM to aggregate the data

In [37]:
sql_product_totals = """
    SELECT products.`productName` AS `Product Name`, 
        SUM(orders.`quantityOrdered`) AS `Total Quantity Ordered`, 
        SUM(orders.`orderTotalPrice`) AS `Total Revenue on Product`
    FROM `{0}`.`fact_orders` AS orders
    INNER JOIN `{0}`.`dim_products` AS products
    ON orders.productName = products.productName
    GROUP BY products.`productName`
    ORDER BY `Total Revenue on Product` DESC;
""".format(dst_dbname)

df_product_totals = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_product_totals)
df_product_totals.head()

,Product Name,Total Quantity Ordered,Total Revenue on Product
0,1992 Ferrari 360 Spider red,624.0,96850.74
1,2001 Ferrari Enzo,318.0,59852.24
2,1952 Alpine Renault 1300,295.0,58416.61
3,1969 Ford Falcon,363.0,57403.47
4,2003 Harley-Davidson Eagle Drag Bike,314.0,54193.69


In [38]:
sql_customer_totals = """
    SELECT customers.`customerName` AS `Customer Name`,
        SUM(orders.`quantityOrdered`) AS `Total Items Ordered`,
        SUM(orders.`orderTotalPrice`) AS `Total Spent`
    FROM `{0}`.`fact_orders` AS orders
    INNER JOIN `{0}`.`dim_customers` AS customers
    ON orders.customerKey = customers.customerKey
    GROUP BY customers.`customerName`
    ORDER BY `Total Spent` DESC;
""".format(dst_dbname)

df_customer_totals = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_customer_totals)
df_customer_totals.head()

,Customer Name,Total Items Ordered,Total Spent
0,Euro+ Shopping Channel,2153.0,189840.15
1,Mini Gifts Distributors Ltd.,1898.0,167783.08
2,"Dragon Souveniers, Ltd.",1452.0,150123.15
3,Muscle Machine Inc,1159.0,117634.88
4,Mini Creations Ltd.,1044.0,93565.24
